# Import Libraries

In [1]:
import sys
sys.path.append("/workspaces/h2ogpt")
sys.path.append("/workspaces/h2ogpt/src")
from langchain_community.document_loaders import PyPDFLoader
import types
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from src.gpt_langchain import prep_langchain,get_existing_db
from src.utils_langchain import _chunk_sources
from src.gpt4all_llm import H2OLlamaCpp
from src.utils import FakeTokenizer
from langchain_community.vectorstores import FAISS
from src.prompter import Prompter
from src.gpt_langchain import get_template, get_docs_with_score, select_docs_with_score, split_merge_docs

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
def clean_doc(docs1):
    if not isinstance(docs1, (list, tuple, types.GeneratorType)):
        docs1 = [docs1]
    for doci, doc in enumerate(docs1):
        docs1[doci].page_content = '\n'.join([x.strip() for x in doc.page_content.split("\n") if x.strip()])
    return docs1


# Load PDF

In [3]:
pdf_doc = PyPDFLoader("../Adjuster Report.PDF").load()
docs = [x for x in pdf_doc if x.page_content]
docs = clean_doc(docs)
docs = _chunk_sources(docs,db_type='faiss')
print(len(docs))

65


# Load Embedding Model

In [4]:
hf_embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
hf_embedding_model.client.eval()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# Prepare db

In [5]:
db = FAISS.from_documents(docs, hf_embedding_model)

# Prepare Model

In [6]:
prompt_type = 'llama2'
prompt_dict=''
system_prompt = 'Give me a summary that is well-structured yet consice.'
prompter = Prompter(prompt_type, prompt_dict, debug=False, stream_output=False,
                            system_prompt=system_prompt)

model = H2OLlamaCpp(
    model_path='../llamacpp_path/llama-2-7b-chat.Q6_K.gguf',
    n_ctx=2048,
    temperature=0.1,
    top_p=0.7,
    prompter=prompter

)

inner_tokenizer = FakeTokenizer(tokenizer=model.client, is_llama_cpp=True, model_max_length=4096)

Failed to listen to n_gpus: No module named 'llama_cpp_cuda'


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../llamacpp_path/llama-2-7b-chat.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32 

In [25]:
query = 'Where is the location of accident?'
iinput = ''
pre_prompt_query = 'Pay attention and remember the information below, which will help to answer the question or imperative after the context ends.'
prompt_query = 'According to only the information in the document sources provided within the context above, write an insightful and well-structured response to: '
pre_prompt_summary = 'In order to write a concise single-paragraph or bulleted list summary, pay attention to the following text.'
prompt_summary = 'Using only the information in the document sources above, write a condensed and concise summary of key results (preferably as bullet points).'
langchain_action = 'Extract'
query_action = False
summarize_action = True
auto_reduce_chunks = False
add_search_to_context = False
system_prompt = 'Give a summary that is well-structured yet concise.'
doc_json_mode = False

template, template_if_no_docs, auto_reduce_chunks, query = \
        get_template(query, iinput,
                     pre_prompt_query, prompt_query,
                     pre_prompt_summary, prompt_summary,
                     langchain_action,
                     query_action,
                     summarize_action,
                     True,  # just to overestimate prompting
                     auto_reduce_chunks,
                     add_search_to_context,
                     system_prompt,
                     doc_json_mode,
                     prompter=prompter)

# Similarity Score

In [26]:
query_embedding = query
k_db = 10
filter_kwargs = {}
filter_kwargs_backup = {}
db_type = 'faiss'
text_context_list = []
chunk_id_filter = None
where_document_dict = {}
verbose = False
top_k_docs = 10
one_doc_size = None
tokenizer = inner_tokenizer
max_input_tokens = 3840
docs_token_handling = 'split_or_merge'
docs_joiner = '\n\n'

docs_with_score = get_docs_with_score(query_embedding, k_db,
                                                  filter_kwargs,
                                                  filter_kwargs_backup,
                                                  db, db_type,
                                                  text_context_list=text_context_list,
                                                  chunk_id_filter=chunk_id_filter,
                                                  where_document_dict=where_document_dict,
                                                  verbose=verbose)

docs_with_score = select_docs_with_score(docs_with_score, top_k_docs, one_doc_size)

docs_with_score, max_doc_tokens = split_merge_docs(docs_with_score,
                                                           tokenizer,
                                                           max_input_tokens=max_input_tokens,
                                                           docs_token_handling=docs_token_handling,
                                                           joiner=docs_joiner,
                                                           verbose=verbose)


In [27]:
# update template in case situation changed or did get docs
    # then no new documents from database or not used, redo template
    # got template earlier as estimate of template token size, here is final used version'

iinput = ''
pre_prompt_query = 'Pay attention and remember the information below, which will help to answer the question or imperative after the context ends.'
prompt_query = 'According to only the information in the document sources provided within the context above, write an insightful and well-structured response to: '
pre_prompt_summary = 'In order to write a concise single-paragraph or bulleted list summary, pay attention to the following text.'
prompt_summary = 'Using only the information in the document sources above, write a condensed and concise summary of key results (preferably as bullet points).'
got_any_docs = True

template, template_if_no_docs, auto_reduce_chunks, query = \
    get_template(query, iinput,
                    pre_prompt_query, prompt_query,
                    pre_prompt_summary, prompt_summary,
                    langchain_action,
                    query_action,
                    summarize_action,
                    got_any_docs,
                    auto_reduce_chunks,
                    add_search_to_context,
                    system_prompt,
                    doc_json_mode,
                    prompter=prompter)

In [28]:
docs_with_score

[(Document(page_content='5\nLOCATION OF ACCIDENT\nLocation  : At KM 24 Jalan Bahau – Kemayan, 72120 Jempol,\nNegeri Sembilan.\nType of Carriageway   : Double carriageway of bitumen surface .\nTraffic Flow    : Two direction s.\nOne lane, in the direction Jerantut.\nOne lane, in the direction to  Serting .\nCentre Demarcation   :         Double continuous white lines, separating opposing flow of\ntraffic.\nWidth of Lanes   : About 3m each .\nRoad Signs    : Destinations/Directions.\nVisibility    : Good.\nLighting    : Streetlights .\n\n7\nPhoto of scene of accident, facing  Jerantut .\nCustodian ’s direction from  his residen ce at No. 13, Lorong 1, Jalan TM 3A, Taman\nTanjung Minyak, 75250 Melaka to Taman Damak Utama, 27030 Damak,\nJerantut, Pahang.\nPhoto of scene of accident , facing  from Serting .\nThird Party Driver ’s direction from  his residence at No. 253 Blok 11, Felda Lui Timur ,\n72120 Bandar Seri Jempol, Negeri Sembilan  heading to grocery store  at Serting.\n\nRoad Condi

In [29]:
from langchain.prompts import PromptTemplate

# Create a prompt template with input variables
prompt_template = PromptTemplate(
    input_variables=["text"],
    template=template
)

# Format the prompt template with a value for the input variable
prompt = prompt_template.format(text=docs_with_score[0][0].page_content)

In [30]:
answer = model._call(prompt=prompt)

Llama.generate: prefix-match hit



llama_print_timings:        load time =    1603.19 ms
llama_print_timings:      sample time =      44.00 ms /   256 runs   (    0.17 ms per token,  5818.71 tokens per second)
llama_print_timings: prompt eval time =  292141.93 ms /  1604 tokens (  182.13 ms per token,     5.49 tokens per second)
llama_print_timings:        eval time =   64301.72 ms /   255 runs   (  252.16 ms per token,     3.97 tokens per second)
llama_print_timings:       total time =  357672.55 ms /  1859 tokens


In [31]:
print(answer)

  Of course! Here's a condensed and concise summary of key results based on the provided document sources:
• Location of accident: KM 24 Jalan Bahau – Kemayan, Negeri Sembilan.
• Type of carriageway: Double carriageway of bitumen surface.
• Traffic flow: Two directions, one lane in each direction.
• Centre demarcation: Double continuous white lines separating opposing flow of traffic.
• Width of lanes: About 3m each.
• Road signs: Destinations/Directions.
• Visibility: Good.
• Lighting: Streetlights.
• Weather: Fine.
• Time of accident: Mild traffic volume.
• Custodian's direction: In the direction of Taman Damak Utama, 27030 Damak, Jerantut, Pahang.
• Third Party Driver's direction: From his residence at No. 253 Blok 11, Felda Lui Timur, 72120 Bandar Seri Jempol, Negeri Sembilan, heading to
